Pre-import modules and setup filepaths

In [1]:
%matplotlib ipympl
import simulationlib as sl
import lasinglib as ll
import measurelib as ml

import modulators as mo
import shapes

import numpy as np
import matplotlib.pyplot as plt

import logging
import os

from matplotlib import cm

from scipy.optimize import curve_fit

from tqdm.notebook import tqdm
import multiprocess

from scipy.special import erf

from IPython.display import HTML

os.chdir(os.path.abspath(''))

logging.basicConfig()
logging.getLogger().setLevel(logging.WARNING)

Initialize simulation framework and useful constants

In [2]:
SILICON = sl.Material(diffusivity=88, emissivity=0.09, specific_heat=0.7, density=0.002329002)
CHIP = sl.SimGrid(32, 97, 0.03, use_spar=False,
                  spar_thickness=0.5, spar_width=1)

sim = sl.Simulation(CHIP, SILICON, duration=4, pulses=None, ambient_temp=300,
                    starting_temp=303, neumann_bc=False, 
                    edge_derivative=0, sample_framerate=0, intended_pbs=1,
                    dense_logging=False, timestep_multi=1, radiation=True, progress_bar=True, silent=True)

print(sim.TIMESTEP)

EXPERIMENTAL_SIGMA = 0.769941159 # beam sigma, mm

CENTER = (CHIP.CENTERPOINT)

0.00030918173122445626


In [3]:
# hotspot temp sim
ROI_max_temps = []
powers = np.linspace(1, 9, 9)
pos = (16, 16)
duration = 3

op = []

def coupled_exp(t):
    grow_const = 0.5
    cutoff_time = np.log(2) / grow_const
    if t < cutoff_time:
        output = np.exp(grow_const * t) - 1
    else:
        output = np.exp(-(t - cutoff_time))
    op.append(output)
    return output

def double_exp(t, )

for p in powers:
    print(f"power: {p}")
    pulse = ll.LaserPulse(CHIP, start=0, duration=duration, position=pos, power=p, modulators=[coupled_exp],
                          sigma=EXPERIMENTAL_SIGMA, measure_target=True, measure_tag="ROIpulse", target_modes=["MAX"], measure_padding=(0, 5.5))
    sim.pulses = [pulse]
    result = sim.simulate(analyzers=[])
    ROI_max_temps.append(result['ROIpulse MAX 0'])

time = result['ROIpulse time']

SyntaxError: expected ':' (770296154.py, line 19)

In [ ]:
plt.close(1)
color = iter(cm.rainbow(np.linspace(0, 1, len(powers))))

fig1, ax1 = plt.subplots(num=1)

for num, (c, power) in enumerate(zip(color, powers)):
    ax1.plot(time, ROI_max_temps[num], color=c)

ax1.plot(time, op[:len(time)], color="black")

plt.show()

In [ ]:
plt.close(2)
fig2, ax2 = plt.subplots(num=2)
color = iter(cm.rainbow(np.linspace(0, 1, len(powers))))

slopeind = 4000

slopes = []

for num, (c, power) in enumerate(zip(color, powers)):
    grad = np.gradient(ROI_max_temps[num])
    ax2.plot(time, grad, color=c)
    slopes.append(grad[slopeind])
plt.show()

In [ ]:
plt.close(3)
fig3, ax3 = plt.subplots(num=3)

ax3.plot(powers, np.abs(slopes), marker=".")
plt.show()